# Handwritten Digit Recognition
### Deep learning model to recognize handwritten digit
### Dataset: MNIST Dataset

## 1. Import the necessary libraries

In [34]:
import os
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras import backend

## 2. Load the MNIST dataset

## Function to print the data details
### Input: <ol><li>x_train - Training Data</li><li>x_test  - Testing Data</li></ol>
### Output: None 

In [19]:
def print_data_details(x_train, x_test):
    print('*' * 10 + ' Training Data Details ' + '*' * 10)
    print(f'=> Number of Training Samples: {x_train.shape[0]}')
    print(f'=> Shape of Training Data: {x_train.shape}')
    print('*' * 10 + ' Testing Data Details ' + '*' * 10)
    print(f'=> Number of Testing Samples: {x_test.shape[0]}')
    print(f'=> Shape of Testing Data: {x_test.shape}')

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print_data_details(x_train, x_test)

********** Training Data Details **********
=> Number of Training Samples: 60000
=> Shape of Training Data: (60000, 28, 28)
********** Testing Data Details **********
=> Number of Testing Samples: 10000
=> Shape of Testing Data: (10000, 28, 28)


## 3. Process the data
### Preprocessing is required to be performed on the data to normalize and refine the data suitable for the neural network's functioning.
### Following steps are performed in the preprocessing stage
<ol>
<li>Reshaping the data to contain one extra dimension for the requirement suitable for the CNN model</li>
<li>Convert class vectors to binary class matrices</li>
<li>Typecast the data to float type and rescale it from 0 - 255 scale to 0 - 1 scale</li>
</ol>

In [21]:
input_shape = (28, 28, 1)
num_classes = 10

# Step 1
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# Step 2
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Step 3
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print('*' * 10 +' After Proprocessing: Data Details' + '*' * 10)
print_data_details(x_train, x_test)

********** After Proprocessing: Data Details**********
********** Training Data Details **********
=> Number of Training Samples: 60000
=> Shape of Training Data: (60000, 28, 28, 1)
********** Testing Data Details **********
=> Number of Testing Samples: 10000
=> Shape of Testing Data: (10000, 28, 28, 1)


## 4. Create the Convolution Neural Network

In [24]:
batch_size = 128
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=categorical_crossentropy, optimizer=Adadelta(), metrics=['accuracy'])

## 5. Training the model
### After training, save the weights and model definition

In [25]:
history = model.fit(x_train, 
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 57s 120ms/step - loss: 2.2980 - accuracy: 0.1199 - val_loss: 2.2411 - val_accuracy: 0.3752
Epoch 2/10
469/469 [==============================] - 56s 120ms/step - loss: 2.2329 - accuracy: 0.2632 - val_loss: 2.1589 - val_accuracy: 0.6028
Epoch 3/10
469/469 [==============================] - 54s 115ms/step - loss: 2.1513 - accuracy: 0.3951 - val_loss: 2.0460 - val_accuracy: 0.6784
Epoch 4/10
469/469 [==============================] - 52s 110ms/step - loss: 2.0374 - accuracy: 0.5024 - val_loss: 1.8823 - val_accuracy: 0.7286
Epoch 5/10
469/469 [==============================] - 58s 123ms/step - loss: 1.8766 - accuracy: 0.5700 - val_loss: 1.6593 - val_accuracy: 0.7658
Epoch 6/10
469/469 [==============================] - 57s 121ms/step - loss: 1.6681 - accuracy: 0.6168 - val_loss: 1.4004 - val_accuracy: 0.7839
Epoch 7/10
469/469 [==============================] - 55s 117ms/step - loss: 1.4412 - accuracy: 0.6545 - val_loss: 1.1543 - val_ac

In [33]:
if('models' not in os.listdir()):
    os.mkdir('models')
model.save(f'models/mnist_ep{epochs}_batchs{batch_size}.h5')

## 6. Evaluate the model
### Test how well the model works with the test data

In [35]:
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Loss: {score[0]}')
print(f'Test Accuracy: {score[1]}')

Test Loss: 0.7194458246231079
Test Accuracy: 0.84170001745224


In [3]:
from tensorflow.keras.models import load_model
import tkinter as tk
from PIL import ImageGrab, Image
import numpy as np

In [4]:
model = load_model('models/mnist_ep10_batchs128.h5')

In [5]:
def predict_digit(model, image):
    image = image.reshape((28, 28))
    image = image.convert('L')
    image = np.array(image)
    image = image.reshape(1, 28, 28, 1)
    image = image / 255.0
    result = model.predict([image])[0]
    return np.argmax(res), max(res)

In [ ]:
class App(tk.TK):
    def __init__(self):
        super().__init__()
        self.x = self.y = 0
        self.canvas = tk.Canvas(self, 
                                width=300, 
                                height=300, 
                                bg="White",
                                cursor="cross")
        self.label = tk.Label(self, 
                                text="Predicting...",
                                font=("Helvetica, 48"))
        self.classify_btn = tk.Button(self,
                                        text="Recognise",
                                        command=self.classify_handwriting)
        self.button_clear = tk.Button(self,
                                        text="Clear", command=self.clear_all)
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo

